xgboost


In [9]:

import numpy as np 
import pandas as pd 
import seaborn as sb
import xgboost as xgb

from sklearn import metrics

In [10]:
raw_train_data = pd.read_csv("D:\\train.csv")
raw_test_data = pd.read_csv("D:\\test.csv")

In [ ]:
print(raw_train_data.shape)
print(raw_test_data.shape)

In [ ]:
Train_data = raw_train_data.iloc[:, 1:95]
labels = set(Train_data.iloc[:,93:94].target)
Valid_x = np.array(raw_test_data.iloc[:, 1:94])
Total_x = np.array(Train_data.iloc[:,0:93])
Total_y = np.array(Train_data.iloc[:,93:94])